In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import sys
import csv
from batchmake import Batcher
import astwalker
import ast
import tokenize
from io import StringIO

In [2]:
pad_token, pad_id = "§PAD§", 0
oov_token, oov_id = "§OOV§", 1
indent_token = "§<indent>§"
dedent_token = "§<dedent>§"
number_token = "§NUM§"

In [4]:
datas= 'data_samples/'
word_to_id='data_samples/mapping.map'
with open(word_to_id, 'rb') as f:
    word_to_id = pickle.load(f)
inv_map = {v:k for k, v in word_to_id.items() }
batch_data = datas+'output.txt.part0'
# batch_data = datas+'preprocess.part0'
with open(batch_data, 'rb') as f:
    picked_data = pickle.load(f)
    
print(len(picked_data))
train_proportion = 0.5
valid_proportion = 0.2    

2515


In [5]:
subdirectories = [os.path.join(datas,o) for o in os.listdir(datas) if os.path.isdir(os.path.join(datas,o))]

In [6]:
python_files = [(directory, [y for x in os.walk(directory) for y in iglob(os.path.join(x[0], '*.py'))])
                for directory in subdirectories]

In [7]:
train_split = int(len(python_files) * train_proportion)
valid_split = train_split + int(len(python_files) * valid_proportion)
train_files = []
valid_files = []
test_files = []

In [8]:
for project in python_files[:train_split]:
    train_files.extend([f[len(datas):] for f in project[1]])
for project in python_files[train_split:valid_split]:
    valid_files.extend([f[len(datas):] for f in project[1]])
for project in python_files[valid_split:]:
    test_files.extend([f[len(datas):] for f in project[1]])

In [9]:
def write_to_file(fname, lst):
    with open(os.path.join(datas, fname), "w") as write_file:
        for f in lst:
            print(f, file=write_file)

In [10]:
write_to_file("train_files.txt", train_files)
write_to_file("valid_files.txt", valid_files)
write_to_file("test_files.txt", test_files)

### 데이터셋을 train, valid, test셋으로 만든다.

### 파일 읽기

In [11]:
python_files = [os.path.join(datas, f) for f in train_files]
mapping = (lambda x: x) if word_to_id is None else (lambda x: word_to_id.get(x, oov_id))
def get_source_tree(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        fstr = f.read()
    fstr = fstr.replace('\r\n', '\n').replace('\r', '\n')
    if not fstr.endswith('\n'):
        fstr += '\n'
    return fstr, ast.parse(fstr, filename=filename)

In [12]:
def preprocess(tokentype, tokenval):
    if tokentype == tokenize.NUMBER:
        return number_token

    elif tokentype == tokenize.INDENT:
        return indent_token

    elif tokentype == tokenize.DEDENT:
        return dedent_token

    return tokenval

In [13]:
data = []
definition_positions = []
identifier_usage = []
gen_def_positions = True
error_data = 0
data_file_name = []
for filename in python_files:
    try:
        source, tree = get_source_tree(filename)
        tokens = tokenize.generate_tokens(StringIO(source).readline)
        data.append([(mapping(preprocess(tokenType, tokenVal)), start) for tokenType, tokenVal, start, _, _
                     in tokens
                     if tokenType != tokenize.COMMENT and
                     not tokenVal.startswith("'''") and
                     not tokenVal.startswith('"""') and 
                     (tokenType == tokenize.DEDENT or tokenVal != "")])
        data_file_name.append(filename)
        if gen_def_positions:
            walker = astwalker.ASTWalker()
            walker.walk(tree)
            definition_positions.append(walker.definition_positions)
            identifier_usage.append(walker.name_usage)
    except:
        error_data+=1
        continue
#         print("Error when tokenizing %s: %s" % (filename, sys.exc_info()[:3]))

### 전체 파일 ( python_files ) 중에서 에러난거 뺀데이터 (data) 

In [14]:
len(python_files),error_data,len(data), len(definition_positions)

(3675, 41, 3634, 3634)

### 데이터 비교

In [15]:
d , dd=get_source_tree(data_file_name[200])
print(d)

from __future__ import unicode_literals
import binascii
import os
from django.contrib.auth.models import User
from django.core.validators import MinLengthValidator
from django.db import models
from django.utils.encoding import python_2_unicode_compatible
from django.utils import timezone


@python_2_unicode_compatible
class Class385(models.Model):
    '\n    An API token used for user authentication. This extends the stock model to allow each user to have multiple tokens.\n    It also supports setting an expiration time and toggling write ability.\n    '
    var3736 = models.ForeignKey(User, related_name='tokens', on_delete=models.CASCADE)
    var2751 = models.DateTimeField(auto_now_add=True)
    var215 = models.DateTimeField(blank=True, null=True)
    var2585 = models.CharField(max_length=40, unique=True, validators=[MinLengthValidator(40)])
    var2200 = models.BooleanField(default=True, help_text='Permit create/update/delete operations using this key')
    var1960 = models.CharField

In [16]:
token_to_str = []
for tokend_data,position in data[200]:
     token_to_str.append(inv_map[tokend_data])
print(' '.join(token_to_str))

from __future__ import unicode_literals 
 import binascii 
 import os 
 from django . contrib . auth . models import User 
 from django . core . validators import MinLengthValidator 
 from django . db import models 
 from django . utils . encoding import python_2_unicode_compatible 
 from django . utils import timezone 
 
 
 @ python_2_unicode_compatible 
 class Class385 ( models . Model ) : 
 §<indent>§ §OOV§ 
 var3736 = models . ForeignKey ( User , related_name = §OOV§ , on_delete = models . CASCADE ) 
 var2751 = models . DateTimeField ( auto_now_add = True ) 
 var215 = models . DateTimeField ( blank = True , null = True ) 
 var2585 = models . CharField ( max_length = §NUM§ , unique = True , validators = [ MinLengthValidator ( §NUM§ ) ] ) 
 var2200 = models . BooleanField ( default = True , help_text = §OOV§ ) 
 var1960 = models . CharField ( max_length = §NUM§ , blank = True ) 
 
 
 class Class42 : 
 §<indent>§ var2956 = [ ] 
 
 §<dedent>§ def __str__ ( self ) : 
 §<indent>§ return 

### 원본데이터와 토큰데이터

In [17]:
type_max_rands = {
    "var": 4750, "function": 2900, "Class": 440, "attribute": 2400, "arg": 2400
}

identifier_types = [key for key in type_max_rands]
def_positions = [[[t[1] for t in fp if t[0] == k] for k in identifier_types] for fp in definition_positions]
# ex) def_positions[101] #function, var, class, attribute, arg

In [18]:
inputs = []
line_data_list = []
for line_data in data:
    for i, token in enumerate(line_data):
#         print(token[0])
#         inputs.append(token[0])
        line_data_list.append(token[0])
#         inputs.append(line_data_list)
#         line_data_list=[]
            
#             print(token[0])
        

In [19]:
# for row in line_data_list:
#     print(row)
print(len(line_data_list))

2716373


In [20]:
def make_dataset(sequence, labels):
    ex = tf.train.SequenceExample()
    sequence_length = len(sequence)
    ex.context.feature['length'].int64_list.value.append(sequence_length)
    fl_tokens = ex.feature_lists.feature_list['tokens']
    fl_labels = ex.feature_lists.feature_list['labels']
    for token, label in zip(sequence, labels):
        fl_tokens.feature.add().int64_list.value.append(token)
        fl_labels.feature.add().int64_list.value.append(label)
    return ex

In [22]:
sequence_length=5

token = line_data_list[:sequence_length]
label = line_data_list[1:sequence_length+1]
ex = make_dataset(token,label)

In [23]:
# with open('inputs.csv', 'w',newline='', encoding='utf-8') as csvfile:
#     spamwriter = csv.writer(csvfile)
#     for row_line in inputs:
#         spamwriter.writerow(row_line)
# with open('inputs.csv', 'w',newline='', encoding='utf-8') as csvfile:
#     spamwriter = csv.writer(csvfile)
#     for row_line in inputs:
#     spamwriter.writerow(line_data_list)
with open('inputs', 'w',newline='', encoding='utf-8') as inputs:
    writer = tf.python_io.TFRecordWriter(inputs.name)
    for i in range(len(line_data_list)):
        token = line_data_list[i* sequence_length: (i+1) * sequence_length]
        label = line_data_list[(i* sequence_length)+1: (i+1) * sequence_length+1 ]
        ex = make_dataset(token, label)
        writer.write(ex.SerializeToString())
    writer.close()

### 데이터 만들고 불러오는 예제.

In [ ]:
file_name_queue = tf.train.string_input_producer(['inputs'])
reader = tf.TFRecordReader()
_, data = reader.read(file_name_queue)

context_features = {
    "length": tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features = {
    "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
    "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
}

context_parsed, sequence_parsed = tf.parse_single_sequence_example(serialized=data,
                                 context_features=context_features,
                                 sequence_features=sequence_features
                                )

s = tf.Session()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=s, coord=coord)
s.run(tf.global_variables_initializer())


In [ ]:
length_,tokens_ = s.run([context_parsed,sequence_parsed])
print(length_)
print(tokens_['tokens'], tokens_['labels'])

### target은 input+1 의 인덱스로 하고, sequence는 실행시간에 정함.